In [2]:
from datasets import load_dataset

ds = load_dataset("naver-clova-ix/cord-v2")
image = ds["train"]["image"][0]

README.md:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

(…)-00000-of-00004-b4aaeceff1d90ecb.parquet:   0%|          | 0.00/490M [00:00<?, ?B/s]

(…)-00001-of-00004-7dbbe248962764c5.parquet:   0%|          | 0.00/441M [00:00<?, ?B/s]

(…)-00002-of-00004-688fe1305a55e5cc.parquet:   0%|          | 0.00/444M [00:00<?, ?B/s]

...

(…)-00000-of-00001-cc3c5779fe22e8ca.parquet:   0%|          | 0.00/242M [00:00<?, ?B/s]

(…)-00000-of-00001-9c204eb3f4e11791.parquet:   0%|          | 0.00/234M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/800 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [6]:
import re

from transformers import DonutProcessor, VisionEncoderDecoderModel
from datasets import load_dataset
import torch

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")

device = "cuda"
model.to(device)

# prepare decoder inputs
task_prompt = "<s_cord-v2>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

pixel_values = processor(image, return_tensors="pt").pixel_values

outputs = model.generate(
    pixel_values.to(device),
    decoder_input_ids=decoder_input_ids.to(device),
    max_length= 1000,
    pad_token_id= processor.tokenizer.pad_token_id,
    eos_token_id= processor.tokenizer.eos_token_id,
    use_cache = True,
    bad_words_ids=[[processor.tokenizer.unk_token_id]],
    return_dict_in_generate = True,
)

sequence = processor.batch_decode(outputs.sequences)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
sequence = re.sub(r"<.*?>", "", sequence, count=1).strip() 
print(processor.token2json(sequence))

Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    1280,
    960
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "path_norm": true,
  "qkv_bias": true,
  "transformers_version": "4.47.0",
  "use_absolute_embeddings": false,
  "window_size": 10
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "add_

{'menu': [{'nm': 'Nasi Campur Bali', 'cnt': '1 x', 'price': '75,000'}, {'nm': 'Bbk Bengil Nasi', 'cnt': '1 x', 'price': '125,000'}, {'nm': 'MilkShake Starwb', 'cnt': '1 x', 'price': '37,000'}, {'nm': 'Ice Lemon Tea', 'cnt': '1 x', 'price': '24,000'}, {'nm': 'Nasi Ayam Dewata', 'cnt': '1 x', 'price': '70,000'}, {'nm': 'Free Ice Tea', 'cnt': '3 x', 'price': '0'}, {'nm': 'Organic Green Sa', 'cnt': '1 x', 'price': '65,000'}, {'nm': 'Ice Tea', 'cnt': '1 x', 'price': '18,000'}, {'nm': 'Ice Orange', 'cnt': '1 x', 'price': '29,000'}, {'nm': 'Ayam Suir Bali', 'cnt': '1 x', 'price': '85,000'}, {'nm': 'Tahu Goreng', 'cnt': '2 x', 'price': '36,000'}, {'nm': 'Tempe Goreng', 'cnt': '2 x', 'price': '36,000'}, {'nm': 'Tahu Telor Asin', 'cnt': '1 x', 'price': '40,000.'}, {'nm': 'Nasi Goreng Samb', 'cnt': '1 x', 'price': '70,000'}, {'nm': 'Bbk Panggang Sam', 'cnt': '3 x', 'price': '366,000'}, {'nm': 'Ayam Sambal Hija', 'cnt': '1 x', 'price': '92,000'}, {'nm': 'Hot Tea', 'cnt': '2 x', 'price': '44,000'},

In [13]:
json_file = processor.token2json(sequence)
json_file

{'menu': [{'nm': 'Nasi Campur Bali', 'cnt': '1 x', 'price': '75,000'},
  {'nm': 'Bbk Bengil Nasi', 'cnt': '1 x', 'price': '125,000'},
  {'nm': 'MilkShake Starwb', 'cnt': '1 x', 'price': '37,000'},
  {'nm': 'Ice Lemon Tea', 'cnt': '1 x', 'price': '24,000'},
  {'nm': 'Nasi Ayam Dewata', 'cnt': '1 x', 'price': '70,000'},
  {'nm': 'Free Ice Tea', 'cnt': '3 x', 'price': '0'},
  {'nm': 'Organic Green Sa', 'cnt': '1 x', 'price': '65,000'},
  {'nm': 'Ice Tea', 'cnt': '1 x', 'price': '18,000'},
  {'nm': 'Ice Orange', 'cnt': '1 x', 'price': '29,000'},
  {'nm': 'Ayam Suir Bali', 'cnt': '1 x', 'price': '85,000'},
  {'nm': 'Tahu Goreng', 'cnt': '2 x', 'price': '36,000'},
  {'nm': 'Tempe Goreng', 'cnt': '2 x', 'price': '36,000'},
  {'nm': 'Tahu Telor Asin', 'cnt': '1 x', 'price': '40,000.'},
  {'nm': 'Nasi Goreng Samb', 'cnt': '1 x', 'price': '70,000'},
  {'nm': 'Bbk Panggang Sam', 'cnt': '3 x', 'price': '366,000'},
  {'nm': 'Ayam Sambal Hija', 'cnt': '1 x', 'price': '92,000'},
  {'nm': 'Hot Tea', '

In [16]:
import json
data_string = json.dumps(json_file)

print(type(data_string))

<class 'str'>


In [19]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")
device = torch.device("cuda")
model.to(device)
task_prefix = """
Your task is to extract the information for the fields provided below.
Extract the information in JSON format according to the following JSON schema:
{
  "$defs": {
    "Invoice": {
      "properties": {
        "base_taxable_amount": {
          "anyOf": [{"type": "string"}, {"type": "null"}],
          "default": null,
          "description": "The base amount that is subject to tax",
          "title": "Base Taxable Amount"
        },
        "item": {
          "anyOf": [{"items": {"type": "string"}, "type": "array"}, {"type": "null"}],
          "default": null,
          "description": "List of items in the invoice",
          "title": "Item"
        }
      },
      "required": ["base_taxable_amount", "item"],
      "title": "Invoice",
      "type": "object"
    }
  }
}
Here is the content that you should extract from:
"""
input_text = task_prefix + data_string
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
output_sequences = model.generate(
    input_ids=inputs["input_ids"].to(device),
    attention_mask=inputs["attention_mask"].to(device),
    max_new_tokens=200, 
    do_sample=False,    
)
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print("\nGenerated Structured Output:")
print(generated_text)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]


Generated Structured Output:
["nm": "Nasi Campur Bali", "cnt": "1 x", "price": "75,000", "nm": "Bbk Bengil Nasi", "cnt": "1 x", "price": "125,000", "nm": "MilkShake Starwb", "cnt": "1 x", "price": "37,000", "nm": "Ice Lemon Tea", "cnt": "1 x", "price": "24,000", "nm": "Nasi Ayam Dewata", "cnt": "1 x", "price": "70,000", "nm":


In [20]:
generated_text

'["nm": "Nasi Campur Bali", "cnt": "1 x", "price": "75,000", "nm": "Bbk Bengil Nasi", "cnt": "1 x", "price": "125,000", "nm": "MilkShake Starwb", "cnt": "1 x", "price": "37,000", "nm": "Ice Lemon Tea", "cnt": "1 x", "price": "24,000", "nm": "Nasi Ayam Dewata", "cnt": "1 x", "price": "70,000", "nm":'